R Code pulled from https://www.statmethods.net/advgraphs/ggplot2.html

In [1]:
#Fig 7
library(ggplot2)
libray(tidyr)
libray(readr)
libray(dplyr)
data = read_csv("F_measure.csv")
chao1_data = read_csv("CHAO1.csv")
data[data[,"Bank"] == "Greengenes","Bank"] = "GG"
data[data[,"Bank"] == "SILVA","Bank"] = "S"

real_chao1 = chao1_data %>%
  filter(`Amplicon / Dataset` == "real")

chao1_data[chao1_data[,"Bank"] == "Greengenes","Bank"] = "GG"
chao1_data[chao1_data[,"Bank"] == "SILVA","Bank"] = "S"
chao1_data = chao1_data %>%
  filter(`Amplicon / Dataset` == "200(V3)") %>%
  filter(Throughput == 50000) %>%
  filter(Complexity == "HC") %>%
  filter(`Error simulation` == "Yes") %>%
  unite(Pipeline, Pipeline, Bank) %>%
  rename("Estimated_CHAO1" = "CHAO1")
chao1_data = cbind(chao1_data, real_chao1[,"CHAO1"])
chao1_data = chao1_data %>%
  mutate(Chao1_Percent = round(((Estimated_CHAO1 - CHAO1)/CHAO1)*100,1) )

data = data %>%
  filter(Amplicon == "200(V3)") %>%
  filter(Throughput == 50000) %>%
  filter(Complexity == "HC") %>%
  filter(`Error simulation` == "Yes") %>%
  unite(Pipeline, Pipeline, Bank)
data[data[,"Pipeline"] == "BMP_GG","Pipeline"] = "* BMP_GG"
data[data[,"Pipeline"] == "mothur_S","Pipeline"] = "* mothur_S"
data[data[,"Pipeline"] == "One Codex_One Codex 28k","Pipeline"] = "* One Codex_One Codex 28k"
data[data[,"Pipeline"] == "QIIME SortMeRna SUMACLUST_GG","Pipeline"] = "* QIIME SortMeRna SUMACLUST_GG"
data[data[,"Pipeline"] == "CLARK_RefSeq Bacteria","Pipeline"] = "CLARK"
data[data[,"Pipeline"] == "kraken_MiniKraken DB","Pipeline"] = "Kraken"

CHAO1_Percent = chao1_data$Chao1_Percent
data = cbind(data, CHAO1_Percent)
data = data[!data[,"Pipeline"] == "QIIME UCLUST_GG",]
data = data[!data[,"Pipeline"] == "QIIME UCLUST_S",]
ggplot(data, aes(x = Pipeline, y = `F-measure`, fill = Rank)) +
  geom_bar(stat = "identity") +
  scale_x_discrete(limits = c("* BMP_GG","BMP_S","CLARK","Kraken","* mothur_S","mothur_GG","One Codex_RefSeq 65","* One Codex_One Codex 28k","One Codex_S","* QIIME SortMeRna SUMACLUST_GG","QIIME SortMeRna SUMACLUST_S")) +
  facet_wrap(~Rank, ncol = 2) +
  geom_text(size = 2, aes(label=CHAO1_Percent, y = 1)) +
  geom_errorbar(aes(ymin=Recall, ymax=Precision)) +
  theme(axis.text.x = element_text(angle = 90), legend.position = "none") +
  ylab("F-measure") +
  coord_cartesian(ylim= c(0.4, 1))
